In [2]:
import findspark
findspark.init('/opt/anaconda3/envs/pyspark_env/lib/python3.10/site-packages/pyspark')  # Replace if needed


In [3]:
from pyspark.sql import SparkSession;

In [4]:
#Initiaize Spark session
spark = SparkSession.builder.appName("Using SQL with Spark").getOrCreate()

25/07/03 13:36:45 WARN Utils: Your hostname, Anirbans-iMac.local resolves to a loopback address: 127.0.0.1; using 192.168.0.132 instead (on interface en0)
25/07/03 13:36:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/03 13:36:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import sys
print("Driver:", sys.version)
print("Worker:", spark.sparkContext.parallelize([0]).map(lambda x: __import__('sys').version).collect()[0])


Driver: 3.10.18 (main, Jun  5 2025, 08:13:51) [Clang 14.0.6 ]


Worker: 3.10.18 (main, Jun  5 2025, 08:13:51) [Clang 14.0.6 ]


In [6]:
data = [("Alice","HR",23),("Jim","IT",28),("Mihir","Finance",38)]
columns = ["Name", "Department", "Age"]

df = spark.createDataFrame(data=data, schema=columns)

In [7]:
df.show()

+-----+----------+---+
| Name|Department|Age|
+-----+----------+---+
|Alice|        HR| 23|
|  Jim|        IT| 28|
|Mihir|   Finance| 38|
+-----+----------+---+



In [8]:
df.createOrReplaceTempView("Employees")

In [9]:
result = spark.sql('''
                        SELECT * FROM Employees WHERE Age > 30
                       ''')

In [10]:
result.show()

+-----+----------+---+
| Name|Department|Age|
+-----+----------+---+
|Mihir|   Finance| 38|
+-----+----------+---+



In [11]:
df = spark.read.csv("salaries.csv",header=True,inferSchema=True)

In [12]:
df.show(5)

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2020|              EN|             FT| Azure Data Engineer|100000|            USD|       100000|                MU|           0|              MU|           S|
|     2020|              EN|             CT|  Staff Data Analyst| 60000|            CAD|        44753|                CA|          50|              CA|           L|
|     2020|              SE|             FT|Staff Data Scientist|164000|            USD|       164000|                US|          50|              US|           M|
|     2020

In [13]:
df.groupBy('work_year').avg('salary').show()

+---------+------------------+
|work_year|       avg(salary)|
+---------+------------------+
|     2023|159284.64961276695|
|     2022|167507.70024125453|
|     2020| 356637.4533333333|
|     2024| 165331.3257725795|
|     2021| 559350.9908256881|
+---------+------------------+



In [14]:
df.createOrReplaceTempView("Employee_salary")

In [15]:
spark.sql('''
                SELECT work_year,
                       avg(salary)
                FROM Employee_salary
                GROUP BY(work_year);
            ''').show()

+---------+------------------+
|work_year|       avg(salary)|
+---------+------------------+
|     2023|159284.64961276695|
|     2022|167507.70024125453|
|     2020| 356637.4533333333|
|     2024| 165331.3257725795|
|     2021| 559350.9908256881|
+---------+------------------+



In [16]:
spark.sql("CREATE DATABASE IF NOT EXISTS employee_db")

DataFrame[]

In [17]:
spark.sql("DROP TABLE IF EXISTS employee_db.employee_salary")

DataFrame[]

In [18]:
df.write \
  .format("parquet") \
  .mode("overwrite") \
  .option("path", "/Users/AnirbanGuha/Documents/GitHub/Python-2/Introduction to PySpark/spark-warehouse/employee_db.db/employee_salary") \
  .saveAsTable("employee_db.employee_salary")

In [19]:
spark.sql("SELECT * FROM employee_salary LIMIT 10").show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2020|              EN|             FT| Azure Data Engineer|100000|            USD|       100000|                MU|           0|              MU|           S|
|     2020|              EN|             CT|  Staff Data Analyst| 60000|            CAD|        44753|                CA|          50|              CA|           L|
|     2020|              SE|             FT|Staff Data Scientist|164000|            USD|       164000|                US|          50|              US|           M|
|     2020

In [20]:
df.limit(10).toPandas()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,EN,FT,Azure Data Engineer,100000,USD,100000,MU,0,MU,S
1,2020,EN,CT,Staff Data Analyst,60000,CAD,44753,CA,50,CA,L
2,2020,SE,FT,Staff Data Scientist,164000,USD,164000,US,50,US,M
3,2020,EN,FT,Data Analyst,42000,EUR,47899,DE,0,DE,L
4,2020,EX,FT,Data Scientist,300000,USD,300000,US,100,US,L
5,2020,MI,CT,Sales Data Analyst,60000,USD,60000,NG,0,NG,M
6,2020,EX,FT,Staff Data Analyst,15000,USD,15000,NG,0,CA,M
7,2020,MI,FT,Business Data Analyst,95000,USD,95000,US,0,US,M
8,2020,EN,FT,Data Analyst,20000,EUR,22809,PT,100,PT,M
9,2020,EN,FT,Data Scientist,43200,EUR,49268,DE,0,DE,S


In [21]:
from IPython.display import display
display(df.limit(10).toPandas())

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,EN,FT,Azure Data Engineer,100000,USD,100000,MU,0,MU,S
1,2020,EN,CT,Staff Data Analyst,60000,CAD,44753,CA,50,CA,L
2,2020,SE,FT,Staff Data Scientist,164000,USD,164000,US,50,US,M
3,2020,EN,FT,Data Analyst,42000,EUR,47899,DE,0,DE,L
4,2020,EX,FT,Data Scientist,300000,USD,300000,US,100,US,L
5,2020,MI,CT,Sales Data Analyst,60000,USD,60000,NG,0,NG,M
6,2020,EX,FT,Staff Data Analyst,15000,USD,15000,NG,0,CA,M
7,2020,MI,FT,Business Data Analyst,95000,USD,95000,US,0,US,M
8,2020,EN,FT,Data Analyst,20000,EUR,22809,PT,100,PT,M
9,2020,EN,FT,Data Scientist,43200,EUR,49268,DE,0,DE,S


In [22]:
import pandas as pd
print(pd.__version__)

2.3.0
